#### Plot the averages of sectors and find correlation

In [1]:
# Include libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import altair as alt

Data to include:
- Dates
- Year
- symbol
- gics_sector_name
- PX_LAST
- DATE_OF_LAST_EXECUTIVE_CHANGE
- TOT_STK_AWD_GIVEN_TO_CEO_EQUIV
- TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV
- TOT_BONUSES_PAID_TO_CEO_EQUIV
- TOT_N_EQT_INCENT_GVN_TO_CEO_EQ
- COMP_LKD_LAST_CEO_&_EQUIV_CHG_DT
- ALL_OTHER_COMP_AW_TO_CEO_EQUIV

The data for the last seven columns are found in the SPX_Data directory

Each CSV file contains yearly data for the named feature. 

Last price every week (PX_LAST) contained in 503PXlast.

### Create dataframe

In [2]:
# Dates, years, symbol, PX_LAST 

# Read ata
px_last_raw_data = pd.read_csv('../../data/503PXlast.csv', index_col=0, parse_dates=True)

# Melt (reshape to long dataframe)
px_last_long = px_last_raw_data.reset_index().melt(id_vars='DATES', var_name='symbol', value_name='price')

# Rename column for consistency
px_last_long.rename(columns={'DATES':'date'},inplace=True)

# Create year column beside date column
px_last_long.insert(1, 'year', pd.to_datetime(px_last_long['date']).dt.year)

display(px_last_long.head())
px_last_long.index

,date,year,symbol,price
0,2015-01-02,2015,LYB UN Equity,80.07
1,2015-01-09,2015,LYB UN Equity,78.98
2,2015-01-16,2015,LYB UN Equity,79.84
3,2015-01-23,2015,LYB UN Equity,80.80
4,2015-01-30,2015,LYB UN Equity,79.09


RangeIndex(start=0, stop=263069, step=1)

In [3]:
# gics_sector_name

# Read data 
gics_sector_data = pd.read_csv('../../data/503_GICSData.csv', index_col=0)

# Merge dataframe with gics_sector_data dataframe
merged_df = px_last_long.merge(gics_sector_data[['gics_sector_name']], left_on='symbol', right_index=True,how='left')

# Move gics_sector_name data beside symbol
merged_df.rename(columns={'gics_sector_name':'temp'}, inplace=True)
merged_df.insert(3, 'gics_sector_name', merged_df['temp'])
merged_df.drop('temp', axis=1, inplace=True)

display(merged_df.head())

,date,year,symbol,gics_sector_name,price
0,2015-01-02,2015,LYB UN Equity,Materials,80.07
1,2015-01-09,2015,LYB UN Equity,Materials,78.98
2,2015-01-16,2015,LYB UN Equity,Materials,79.84
3,2015-01-23,2015,LYB UN Equity,Materials,80.80
4,2015-01-30,2015,LYB UN Equity,Materials,79.09


In [4]:
'''
The data is formatted similarly; use the function to format it such that 
the date and symbols columns can be matched to those of merged_df.

@param: feature name 

'''

def reformat_dataframe(feature, data_name='value'):
    data = pd.read_csv('../../data/SPX_Data/' + feature + '.csv', parse_dates=['Date'])
    long_data = data.melt(id_vars='Date', var_name='symbol', value_name=data_name)
    long_data.rename(columns={'Date':'date'}, inplace=True)
    long_data.insert(1,'year', pd.to_datetime(long_data['date']).dt.year)

    return long_data

In [5]:
# Remaining columns:
#- DATE_OF_LAST_EXECUTIVE_CHANGE        UNUSED
#- TOT_STK_AWD_GIVEN_TO_CEO_EQUIV
#- TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV
#- TOT_BONUSES_PAID_TO_CEO_EQUIV
#- TOT_N_EQT_INCENT_GVN_TO_CEO_EQ
#- ALL_OTHER_COMP_AW_TO_CEO_EQUIV       UNUSED

TOT_STK_AWD_GIVEN_TO_CEO_EQUIV          = reformat_dataframe('TOT_STK_AWD_GIVEN_TO_CEO_EQUIV', data_name='TOT_STK_AWD_GIVEN_TO_CEO_EQUIV')    
TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV          = reformat_dataframe('TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV', data_name='TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV')    
TOT_BONUSES_PAID_TO_CEO_EQUIV           = reformat_dataframe('TOT_BONUSES_PAID_TO_CEO_EQUIV',  data_name='TOT_BONUSES_PAID_TO_CEO_EQUIV')  
TOT_N_EQT_INCENT_GVN_TO_CEO_EQ          = reformat_dataframe('TOT_N_EQT_INCENT_GVN_TO_CEO_EQ', data_name='TOT_N_EQT_INCENT_GVN_TO_CEO_EQ')
ALL_OTHER_COMP_AW_TO_CEO_EQUIV          = reformat_dataframe('ALL_OTHER_COMP_AW_TO_CEO_EQUIV', data_name='ALL_OTHER_COMP_AW_TO_CEO_EQUIV')        


In [8]:
# Finalize main dataframe without featured engineered columns

main_df = pd.merge(merged_df, TOT_STK_AWD_GIVEN_TO_CEO_EQUIV[['year', 'symbol', 'TOT_STK_AWD_GIVEN_TO_CEO_EQUIV']], on=['year', 'symbol'], how='left')
main_df = pd.merge(main_df, TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV[['year', 'symbol', 'TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV']], on=['year', 'symbol'], how='left')
main_df = pd.merge(main_df, TOT_BONUSES_PAID_TO_CEO_EQUIV[['year', 'symbol', 'TOT_BONUSES_PAID_TO_CEO_EQUIV']], on=['year', 'symbol'], how='left')
main_df = pd.merge(main_df, TOT_N_EQT_INCENT_GVN_TO_CEO_EQ[['year', 'symbol', 'TOT_N_EQT_INCENT_GVN_TO_CEO_EQ']], on=['year', 'symbol'], how='left')
main_df = pd.merge(main_df, ALL_OTHER_COMP_AW_TO_CEO_EQUIV[['year', 'symbol', 'ALL_OTHER_COMP_AW_TO_CEO_EQUIV']], on=['year', 'symbol'], how='left')


display(main_df.head())

,date,year,symbol,gics_sector_name,price,TOT_STK_AWD_GIVEN_TO_CEO_EQUIV,TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV,TOT_BONUSES_PAID_TO_CEO_EQUIV,TOT_N_EQT_INCENT_GVN_TO_CEO_EQ,ALL_OTHER_COMP_AW_TO_CEO_EQUIV
0,2015-01-02,2015,LYB UN Equity,Materials,80.07,12356319.0,6518771.0,0.0,4026937.0,20040667.0
1,2015-01-09,2015,LYB UN Equity,Materials,78.98,12356319.0,6518771.0,0.0,4026937.0,20040667.0
2,2015-01-16,2015,LYB UN Equity,Materials,79.84,12356319.0,6518771.0,0.0,4026937.0,20040667.0
3,2015-01-23,2015,LYB UN Equity,Materials,80.80,12356319.0,6518771.0,0.0,4026937.0,20040667.0
4,2015-01-30,2015,LYB UN Equity,Materials,79.09,12356319.0,6518771.0,0.0,4026937.0,20040667.0


In [9]:
# Add cash:equity compensatio ratio column to main_df

main_df.loc[:,"equity_Compensation_CEO"] = main_df['TOT_STK_AWD_GIVEN_TO_CEO_EQUIV']+main_df["TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV"]
main_df.loc[:,"cash_incentive_CEO"] = main_df['TOT_BONUSES_PAID_TO_CEO_EQUIV']+main_df['TOT_N_EQT_INCENT_GVN_TO_CEO_EQ']
main_df.loc[:,"Variable_Pay_CEO"] = main_df["cash_incentive_CEO"]+main_df["equity_Compensation_CEO"]
main_df.loc[:,"cash_incentive_CEO_percent"] = main_df["cash_incentive_CEO"]/main_df["Variable_Pay_CEO"]

In [10]:
# Drop NaNs
main_df.dropna(inplace=True)
main_df.reset_index(inplace=True, drop=True)


In [11]:
# Preview main dataframe 

display(main_df)
main_df.info()

,date,year,symbol,gics_sector_name,price,TOT_STK_AWD_GIVEN_TO_CEO_EQUIV,TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV,TOT_BONUSES_PAID_TO_CEO_EQUIV,TOT_N_EQT_INCENT_GVN_TO_CEO_EQ,ALL_OTHER_COMP_AW_TO_CEO_EQUIV,equity_Compensation_CEO,cash_incentive_CEO,Variable_Pay_CEO,cash_incentive_CEO_percent
0,2015-01-02,2015,LYB UN Equity,Materials,80.07,12356319.0,6518771.0,0.0,4026937.0,20040667.0,18875090.0,4026937.0,22902027.0,0.175833
1,2015-01-09,2015,LYB UN Equity,Materials,78.98,12356319.0,6518771.0,0.0,4026937.0,20040667.0,18875090.0,4026937.0,22902027.0,0.175833
2,2015-01-16,2015,LYB UN Equity,Materials,79.84,12356319.0,6518771.0,0.0,4026937.0,20040667.0,18875090.0,4026937.0,22902027.0,0.175833
3,2015-01-23,2015,LYB UN Equity,Materials,80.80,12356319.0,6518771.0,0.0,4026937.0,20040667.0,18875090.0,4026937.0,22902027.0,0.175833
4,2015-01-30,2015,LYB UN Equity,Materials,79.09,12356319.0,6518771.0,0.0,4026937.0,20040667.0,18875090.0,4026937.0,22902027.0,0.175833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212836,2023-12-01,2023,MOH UN Equity,Health Care,362.76,15500092.0,0.0,0.0,4410000.0,15581723.0,15500092.0,4410000.0,19910092.0,0.221496
212837,2023-12-08,2023,MOH UN Equity,Health Care,367.72,15500092.0,0.0,0.0,4410000.0,15581723.0,15500092.0,4410000.0,19910092.0,0.221496
212838,2023-12-15,2023,MOH UN Equity,Health Care,362.98,15500092.0,0.0,0.0,4410000.0,15581723.0,15500092.0,4410000.0,19910092.0,0.221496
212839,2023-12-22,2023,MOH UN Equity,Health Care,358.38,15500092.0,0.0,0.0,4410000.0,15581723.0,15500092.0,4410000.0,19910092.0,0.221496


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212841 entries, 0 to 212840
Data columns (total 14 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   date                            212841 non-null  datetime64[ns]
 1   year                            212841 non-null  int64         
 2   symbol                          212841 non-null  object        
 3   gics_sector_name                212841 non-null  object        
 4   price                           212841 non-null  float64       
 5   TOT_STK_AWD_GIVEN_TO_CEO_EQUIV  212841 non-null  float64       
 6   TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV  212841 non-null  float64       
 7   TOT_BONUSES_PAID_TO_CEO_EQUIV   212841 non-null  float64       
 8   TOT_N_EQT_INCENT_GVN_TO_CEO_EQ  212841 non-null  float64       
 9   ALL_OTHER_COMP_AW_TO_CEO_EQUIV  212841 non-null  float64       
 10  equity_Compensation_CEO         212841 non-null  float64

In [10]:
# Save finalized dataframe as CSV file
# NOTE: NaN values remain in the dataframe

# main_df.to_csv('../../data/finalized_df.csv')


In [14]:
# Get list of sector names

sectors = main_df['gics_sector_name'].unique()

print(sectors)

['Materials' 'Financials' 'Communication Services' 'Energy'
 'Information Technology' 'Industrials' 'Consumer Staples' 'Health Care'
 'Real Estate' 'Consumer Discretionary' 'Utilities']


In [12]:
# Reference line

# merged_df["marketcap_weight"] = merged_df.groupby(["Date", "gics_sector_name"])["MarketCap"].apply(lambda x: x / x.sum())

In [13]:
# # Ensure data is sorted properly
# df = df.sort_values(by=['symbol', 'date'])

# # Extract the last entry of each year for each symbol
# df_year_end = df.groupby(['symbol', 'year']).last().reset_index()

# # Compute the year-over-year price change as a ratio
# df_year_end['prev_year_price'] = df_year_end.groupby('symbol')['price'].shift(1)
# df_year_end['price_change_ratio'] = df_year_end['price'] / df_year_end['prev_year_price']
# df_year_end = df_year_end.dropna()

In [15]:
# Ensure data is sorted properly after dropping NaNs
main_df.sort_values(by=['gics_sector_name', 'symbol', 'date'], inplace=True)

# Add yearly percentage returns
temp_df = main_df.groupby(['symbol', 'year']).last().reset_index()
temp_df['prev_year_price'] = temp_df.groupby('symbol')['price'].shift(1)
main_df['yearly_percent_return'] = temp_df['price']/temp_df['prev_year_price']



# Add bin category column
num_bins = 10
main_df['compensation_bins'] = pd.cut(main_df['cash_incentive_CEO_percent'], bins=[i/num_bins for i in range(num_bins+1)], include_lowest=True)

display(main_df)
main_df.info()    

,date,year,symbol,gics_sector_name,price,TOT_STK_AWD_GIVEN_TO_CEO_EQUIV,TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV,TOT_BONUSES_PAID_TO_CEO_EQUIV,TOT_N_EQT_INCENT_GVN_TO_CEO_EQ,ALL_OTHER_COMP_AW_TO_CEO_EQUIV,equity_Compensation_CEO,cash_incentive_CEO,Variable_Pay_CEO,cash_incentive_CEO_percent,yearly_percent_return,compensation_bins
68581,2015-01-02,2015,CHTR UW Equity,Communication Services,184.063260,999925.0,8999426.0,0.0,4156600.0,10204787.0,9999351.0,4156600.0,14155951.0,0.293629,NaN,"(0.2, 0.3]"
68582,2015-01-09,2015,CHTR UW Equity,Communication Services,179.141783,999925.0,8999426.0,0.0,4156600.0,10204787.0,9999351.0,4156600.0,14155951.0,0.293629,NaN,"(0.2, 0.3]"
68583,2015-01-16,2015,CHTR UW Equity,Communication Services,178.179606,999925.0,8999426.0,0.0,4156600.0,10204787.0,9999351.0,4156600.0,14155951.0,0.293629,NaN,"(0.2, 0.3]"
68584,2015-01-23,2015,CHTR UW Equity,Communication Services,175.768635,999925.0,8999426.0,0.0,4156600.0,10204787.0,9999351.0,4156600.0,14155951.0,0.293629,NaN,"(0.2, 0.3]"
68585,2015-01-30,2015,CHTR UW Equity,Communication Services,167.125636,999925.0,8999426.0,0.0,4156600.0,10204787.0,9999351.0,4156600.0,14155951.0,0.293629,NaN,"(0.2, 0.3]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115726,2023-12-01,2023,XEL UW Equity,Utilities,61.430000,18000011.0,0.0,0.0,1811160.0,18246008.0,18000011.0,1811160.0,19811171.0,0.091421,NaN,"(-0.001, 0.1]"
115727,2023-12-08,2023,XEL UW Equity,Utilities,60.910000,18000011.0,0.0,0.0,1811160.0,18246008.0,18000011.0,1811160.0,19811171.0,0.091421,NaN,"(-0.001, 0.1]"
115728,2023-12-15,2023,XEL UW Equity,Utilities,61.580000,18000011.0,0.0,0.0,1811160.0,18246008.0,18000011.0,1811160.0,19811171.0,0.091421,NaN,"(-0.001, 0.1]"
115729,2023-12-22,2023,XEL UW Equity,Utilities,62.060000,18000011.0,0.0,0.0,1811160.0,18246008.0,18000011.0,1811160.0,19811171.0,0.091421,NaN,"(-0.001, 0.1]"


<class 'pandas.core.frame.DataFrame'>
Int64Index: 212841 entries, 68581 to 115730
Data columns (total 16 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   date                            212841 non-null  datetime64[ns]
 1   year                            212841 non-null  int64         
 2   symbol                          212841 non-null  object        
 3   gics_sector_name                212841 non-null  object        
 4   price                           212841 non-null  float64       
 5   TOT_STK_AWD_GIVEN_TO_CEO_EQUIV  212841 non-null  float64       
 6   TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV  212841 non-null  float64       
 7   TOT_BONUSES_PAID_TO_CEO_EQUIV   212841 non-null  float64       
 8   TOT_N_EQT_INCENT_GVN_TO_CEO_EQ  212841 non-null  float64       
 9   ALL_OTHER_COMP_AW_TO_CEO_EQUIV  212841 non-null  float64       
 10  equity_Compensation_CEO         212841 non-null  flo

In [16]:
result = main_df.groupby(['gics_sector_name', 'compensation_bins'])['yearly_percent_return'].mean().reset_index()

In [36]:
result.dropna(inplace=True)
for sector_name,i in result.groupby("gics_sector_name"):
    y= i["yearly_percent_return"]
    x=np.arange(len(y))
    np.corrcoef(x,y)[0, 1]
    print("{} {}".format(sector_name,np.corrcoef(x,y)[0, 1]))

Communication Services 0.9999999999999999
Energy 0.7037274116087849
Financials 0.7700653562964468
Industrials 0.39260508080673406
Information Technology 0.9999999999999998
Materials -1.0


In [ ]:
# Save finalized dataframe as CSV file
# NOTE: NaN values remain in the dataframe

# main_df.to_csv('../../data/finalized_df.csv')